In [15]:
library(Hmisc)
library(plyr)
library(dplyr)
library(ggplot2)
library(tidyr)
library(ggpubr)
library(patchwork)
library(pROC)
library(tidymodels)
library(stringr)

In [1]:
#################Function definitions


#function to normalize qPCR using the delta delta Ct method
#data is the data matrix containing raw CT values
#geneNames is the names of the genes to normalize
#normalizationGene is the gene name to use to normalize against commonly this is GAPDH or ACTB
#referenceSamples are the set of sample(s) to use as the reference all other samples will be compared to these samples
###One common approach is to use the set of control samples as the reference

## See this website for a good tutorial on this: https://toptipbio.com/delta-delta-ct-pcr/

deltaDeltaCTCalculation<-function(data, geneNames, normalizationGene, refIndexes, useCancer=FALSE){

    data[data==40]=NA
    deltaCTSamples=data[, geneNames]-data[, normalizationGene]

 
    
    
    DeltaCTReference=vector()
    
    #if set to true than reference sample to index against is the cancer sample with the lowest expression
    #This is helpful for those genes that have no detectable expression in all/most normal samples
    if(useCancer==TRUE)
        {
            crcIndex=which(data$Sample.type=="CRC")
            crcData=data[crcIndex,c(geneNames)]

   #setting samples with no detectable expression to NA we don't want to use them
            #as the reference sample
            crcData[crcData==40]=NA

            #now get row index of max CT for each gene in the CRC dataset
            #row index of max value in each column
            maxCTindex <- apply(crcData, MARGIN = 2, which.max)   

           

            DeltaCTReference[1:length(maxCTindex)]=NA
            for(i in 1:length(maxCTindex)){
                
                DeltaCTReference[i]=crcData[maxCTindex[i], names(maxCTindex)[i]]-crcData[maxCTindex[i], normalizationGene]
            }

  
        }else{
        #calculating the geometric mean of the delta CTs for the reference control samples 

        refData=deltaCTSamples[refIndexes, ]

        
     
        DeltaCTReference=sapply(refData, median, na.rm=TRUE)

    }


    #calculating deltadeltaCT comparing to the reference samples

    deltaDeltaCT=t(t(deltaCTSamples)-DeltaCTReference)

    

    #calculate fold gene expression
    foldGeneExpression=2^(-1*deltaDeltaCT)

  
    #add in sample labels

    temp=cbind(data.frame(Sample_Type=data$Sample_Type), foldGeneExpression)

    foldGeneExpression=temp
    
    return(foldGeneExpression)
    
    }

#small utility function to calculate the geometric mean
geometricMean<-function(data){

    return(exp(mean(log(data), na.rm=TRUE)))
    
}





#####Given a data frame in the data format in which metadata columns contain metadata in the column name
##### and for the columns that contain data the column name is the name of the biomarker being measured
#####produce a violin plot of the data grouping together by disease status

plotViolinPlots<-function(data_long, mainTitle="", ylab="Ct value", dataColumn=3, 
                         ylabBarPlot=c("Fraction of\nSamples without\n Detectable Expression"),
                         ylabScatterPlot=c("Plotted Ct Values\nFor Samples With Detectable\nExpression"), 
                         colorColumn="Sample_Type", removeUndetected=TRUE){


   
data_col_name=colnames(data_long)[dataColumn]


#add another column indicating if data falls below the detection limit
    data_long$detection_limit=rep(TRUE, nrow(data_long))
    
     data_long$detection_limit[which(data_long[, dataColumn]==40)]=FALSE

    data_long$detection_limit[which(data_long[, dataColumn]==log(40))]=FALSE

    
#only plotting points on the violin plot that have been detected
    data_long_above_limit=data_long[which(data_long$detection_limit), ]

    maxValue=max(data_long_above_limit[, dataColumn])
    minValue=min(data_long_above_limit[, dataColumn])

    #defining this here so I can remove the points with undetectable Cts if necassary
    groupSymbols=c("Genes", colorColumn)
    df_fraction_above_detection=data_long %>% group_by_at(groupSymbols) %>% summarize(Fraction_Detected=length(which(detection_limit==TRUE))/length(detection_limit))

    if(removeUndetected){
        
      
        
        if(length(which(data_long$detection_limit==FALSE)) > 0)
        {
            data_long=data_long[-which(data_long$detection_limit==FALSE), ]
        }
    }
    
   
    
  

    #construct violin plot
    plot1=ggplot(data_long, aes(x=Genes, y=.data[[data_col_name]], color=.data[[colorColumn]], shape=.data[[colorColumn]]))+
    geom_point(position = position_jitterdodge(seed = 1, dodge.width = 0.6,  jitter.width=0.1), size=6, 
                           aes(color=.data[[colorColumn]]))+theme_pubr()+theme(text = element_text(size=35), 
                                                          axis.title.x=element_blank(), axis.text.x=element_blank(),
                                                          axis.ticks.x=element_blank(),
                                                          plot.margin = unit(c(0.1,0.1,0,0.1), "cm"),
                                                                        axis.line.x = element_blank())+coord_cartesian(ylim=c(minValue, maxValue))+scale_shape_manual(values = c(15, 16, 17, 18, 20, 21, 22, 23,24))

   
#adding a title    
 plot1=plot1+ggtitle(mainTitle)

      
    
    
    ## bar plot representing the fraction of samples below the limit of detection
    plot2=ggplot(df_fraction_above_detection, aes(x=Genes, y=Fraction_Detected, fill=.data[[colorColumn]])
            )+geom_bar(stat="identity", position=position_dodge(), width = 0.6)+theme_pubr()+theme(text = element_text(size=35), 
                                                                            axis.text.x = 
                                                                             element_text(angle = 90, vjust = 0.5, 
                                                                                      hjust=1, size=35), 
                                                                            axis.text.y = 
                                                                             element_text(size=35), 
                                                                                     legend.position = "none", 
                                                                                                   panel.border = element_rect(colour = "black", fill=NA, size=0.6))
         
     #putting both plots together     
    plot2=plot2+labs(y=ylabBarPlot)+scale_y_continuous( limits = c(0, 1))
    plot1=plot1+labs(y=ylabScatterPlot)

    
   results=plot1/plot2+plot_layout(heights=c(3,1))

    print(results)
}




#A function to calculate the AUC between all possible groups
getAUC<-function(data, refColumns="Normal Colonoscopy", SampleLabels="Sample_Type")
{
    #Collapsing ref samples into a single control group

    Sample_labels=data[,SampleLabels]

    refIndexes=which(Sample_labels %in% refColumns)
    
    Sample_labels[refIndexes]="Control"

    #adding in sample labels
    data[, 1]=Sample_labels

    ##Calculating the AUC for each comparisions
    
    #CRC vs control
    subSetData = data %>% filter(data[, 1] %in% c("CRC", "Control")) 
    CRCvsCont_AUC=apply(subSetData[, 2:ncol(subSetData)], 2, auc, response=subSetData[, 1], quiet=TRUE)

    #AA vs control
    subSetData = data %>% filter(data[, 1] %in% c("AA", "Control")) 
    AAvsCont_AUC=apply(subSetData[, 2:ncol(subSetData)], 2, auc, response=subSetData[, 1], quiet=TRUE)


    #AA+CRC vs control
    subSetData = data %>% filter(data[, 1] %in% c("AA", "CRC", "Control"))   
    caseIndexes=which(subSetData[, 1] %in% c("AA", "CRC"))
    subSetData[caseIndexes, 1]="Case"
    AAplusCRCvsCont_AUC=apply(subSetData[, 2:ncol(subSetData)], 2, auc, response=subSetData[, 1], quiet=TRUE)


    #combine results
    AUCResults=as.data.frame(rbind(CRCvsCont_AUC, AAvsCont_AUC, AAplusCRCvsCont_AUC))
    
    
    return(AUCResults)
  
}


#This function will test genes for statistical significance using mann wilcox and using Fishers exact test 
#if there is a difference between CRC and healthy samples
#multiple testing correction will be done using FDR
#mann witney test may give incorrect results if for most normal samples the gene expression is undefined
#Fishers exact test will work under these circumstances fishers test is used to determine if the proportion
#of cancer samples with detectable expression is higher then the proportion of normal samples with detectable
#expression
#data is the input data
#diseaseGroup is the variable containing the name of the samples correspoding to the disease to be compared
#controlGroup is the variable containing the name of the samples that are the controls to be compared to the disease group                                                                                                   
testDifferentialExpression<-function(data, diseaseGroup=c("CRC"), controlGroup=c("Controls")){

    #collect disease and control samples into seperate groups
    crcIndexes=which(data[,1] %in% diseaseGroup)
    normalIndexes=which(data[,1] %in% controlGroup)

    #initialize the results data frame 
    results=data.frame("Genes"=colnames(data)[2:ncol(data)],"p.value"=rep(NA, ncol(data)-1),  
                       "medianTumor"=rep(NA, ncol(data)-1), "medianNormal"=rep(NA, ncol(data)-1),
                      "meanTumor"=rep(NA, ncol(data)-1), "meanNormal"=rep(NA, ncol(data)-1),
                      "crcPerDetectable"=rep(NA, ncol(data)-1), "normalPerDetectable"=rep(NA, ncol(data)-1), 
                      "Fisher_p.value"=rep(NA, ncol(data)-1) )

    #Go through every gene calculate the p value and other metrics
    for(i in 2:ncol(data)){

       
           #calculte the wilcox p-value 

        
        results$p.value[i-1]= suppressWarnings(wilcox.test(data[crcIndexes, i], data[normalIndexes, i])$p.value)

        #building contigency table to perform fishers test
        #table is count of samples that are above or below the limit of detection 
        #for disease or control samples
        crcPositive=length(which(data[crcIndexes, i]<40))
        crcNegative=length(crcIndexes)-crcPositive
        normalPositive=length(which(data[normalIndexes, i]<40))
        normalNegative=length(normalIndexes)-normalPositive

        contigencyTable=data.frame("crc"=c(crcPositive, crcNegative), 
                                   "normal"=c(normalPositive, normalNegative), 
                                   row.names=c("Positive", "Negative"))

        #caculate fishers p-value
        results$Fisher_p.value[i-1]=fisher.test(contigencyTable)$p.value

        #calculate other metrics
        results$medianTumor[i-1]=median(data[crcIndexes, i])
        results$medianNormal[i-1]=median(data[normalIndexes, i])
        results$meanTumor[i-1]=mean(data[crcIndexes, i])
        results$meanNormal[i-1]=mean(data[normalIndexes, i])
        results$crcPerDetectable[i-1]=length(which(data[crcIndexes, i]<40))/length(crcIndexes)
        results$normalPerDetectable[i-1]=length(which(data[normalIndexes, i]<40))/length(normalIndexes)
    }

#adjust the pvalues for multiple testing
    results$FDR=p.adjust(results$p.value, method = "BH" )
    results$FDR_Fisher=p.adjust(results$Fisher_p.value, method = "BH" )
    
    return(results)
}


#run 5 fold cross validation and return the results. 
runCrossValidation<-function(data, controlGroup=c("Normal", "HP"), numberFolds=5, repeats=1, nameSampleLabelsColumn="Sample_Type"){

     #Collapsing all reference samples into a control group

    Sample_labels=data[,nameSampleLabelsColumn]
    refIndexes=which(Sample_labels %in% controlGroup)
    Sample_labels[refIndexes]="Control"

    caseIndexes=which(!(Sample_labels %in% c("Control")))
    Sample_labels[caseIndexes]="Disease"
    
    
    data[, nameSampleLabelsColumn]=factor(Sample_labels)
    
    colnames(data)[which(colnames(data)==nameSampleLabelsColumn)]="Sample_labels" 


 
    
    # Get the resampling folds
    folds <- vfold_cv(data, v = numberFolds, strata = 'Sample_labels', repeats=repeats)



#Define a model random forest
    model=rand_forest(trees = 2000) %>%
    set_engine("ranger", importance = "impurity") %>%
    set_mode("classification")

    
# Define a workflow
    wflow <- 
      workflow() %>% 
      add_formula(Sample_labels ~ .) %>% 
      add_model(model) 

# Fit the model to the resamples, compute metrics
    results <-
      fit_resamples(
        wflow,
        resamples = folds,
        metrics =  metric_set(roc_auc, sensitivity, specificity),
        #metrics = metric_set(roc_auc),
        control = control_resamples(event_level = 'second', save_pred = TRUE)
      )
    

    return(results)
}



calculateSensandSpecForSingleModel <- function(data, diseaseCategories=c("CRC", "AA"), controlCategories=c("HP", "Normal", "Adenoma"),
                                                labelColumn="Label_Breakdown")
{
    #calculate the sensitivity 
    
    classProbCutoffs=seq(0, 1, by=0.001)
    numberCutoffs=length(classProbCutoffs)

    #initilizing data frame that will be used to store results
    results=data.frame("classProbCutoffs"=classProbCutoffs)
   for(j in 1:length(diseaseCategories)){

        diseaseResultsLable=paste(diseaseCategories[j], "_Sens", sep="")
        results[,  diseaseResultsLable]=rep(NA, numberCutoffs)
       
    }

    results$Specificity=rep(NA, numberCutoffs)


    for(i in 1:numberCutoffs){ 

        cutoff=classProbCutoffs[i]
        
        for(j in 1:length(diseaseCategories))
            {
                diseaseResultsLable=paste(diseaseCategories[j], "_Sens", sep="")
            
                predDisease=data[which(data[, labelColumn] == diseaseCategories[j]), ".pred_Disease"]
                results[i, diseaseResultsLable]=length(which(predDisease > cutoff))/nrow(predDisease)

        
            }

    #getting the disease class probibilities for the Control samples
        predDisease=data[which(pull(data, labelColumn) %in% controlCategories), ".pred_Disease"]
        results$Specificity[i]=1-(length(which(predDisease > cutoff))/nrow(predDisease))
    }

    return(results)
}


#####This function will take as input a dataset of genes plus Ct values a biobank and calculate various performance metrics 
####Biobank must be one of c("Cureline", "BioIVT", "Capital Biosciences", "China_CRO", "China_Collaborator")

getPerformanceMetrics <- function(dataset, bioBank=c("China_CRO"), 
                                  refColumns = c("Normal", "HP"), 
                                  genesToUse=genesTested_POC3, 
                                  sortBy="CRC", 
                                  normalized=FALSE)
{
    if(normalized==FALSE){
    genesToUse=genesToUse
        }

    if(normalized==TRUE){

        genesToUse=2:ncol(dataset)
        }


    if("BioBank" %in% colnames(dataset)){
 
        dataset=dataset %>% filter(BioBank %in% bioBank)
    }





#Calculating AUC for each gene

#correct function call for raw CT data
    aucResultsCRC=getAUC(dataset[, c(which(colnames(dataset)=="Sample_Type"), genesToUse)], refColumns = refColumns, SampleLabels = "Sample_Type")







#get AUC data into a form that can be merged with AUC p-value calculation
    aucResultsCRC=as.data.frame(t(aucResultsCRC))
    aucResultsCRC$Genes=rownames(aucResultsCRC)



#calculating the pvalues and other metrics
    aucPValueCRC=testDifferentialExpression(dataset[, c(which(colnames(dataset)=="Sample_Type"), genesToUse)], diseaseGroup="CRC", controlGroup=refColumns)




#adding AUC to pvalue and other metrics
    aucResultsCRC=merge(aucResultsCRC, aucPValueCRC, by="Genes")





#setting CRC results columns names
    colnames(aucResultsCRC)[5:14]=c("CRC_Mann_pvalue", "CRC_medianTumor", "medianNormal", 
                                            "CRC_meanTumor", "meanNormal", "CRC_%_Detectable",
                                            "Normal_%_Detectable", "CRC_Fisher_pvalue",  
                                           "CRC_Mann_FDR", "CRC_Fisher_FDR")



####### Calculate metrics for AA 

#correct function call of raw CT
    aucPValueAA=testDifferentialExpression(dataset[, c(which(colnames(dataset)=="Sample_Type"), genesToUse)], diseaseGroup="AA", controlGroup=refColumns)






#setting AA results column names
    colnames(aucPValueAA)[2:11]=c("AA_Mann_pvalue", "AA_medianTumor", "medianNormal", 
                                            "AA_meanTumor", "meanNormal", "AA_%_Detectable",
                                            "Normal_%_Detectable", "AA_Fisher_pvalue",  
                                           "AA_Mann_FDR", "AA_Fisher_FDR")




#removing duplicate columns these will be the normal metrics columns these columns were already calculated
#for the CRC versus normal comparision
    aucPValueAA=aucPValueAA[, -c(4, 6, 8)]



####Merging CRC and AA results
    aucResults=merge(aucResultsCRC, aucPValueAA, by="Genes")


#converting AUC for single gene calculation to same format i.e. signal is > 0.5 AUC
    temp=which(aucResults$AAvsCont_AUC < 0.5 )

    if(length(temp>0))
        {
    
        aucResults$AAvsCont_AUC[temp]=1 - aucResults$AAvsCont_AUC[temp]
        }

    temp=which(aucResults$AAplusCRCvsCont_AUC < 0.5 )

    if(length(temp>0))
        {
    
        aucResults$AAplusCRCvsCont_AUC[temp]=1 - aucResults$AAplusCRCvsCont_AUC[temp]

        }


    temp=which(aucResults$CRCvsCont_AUC <0.5 )

    if(length(temp>0))
        {
    
        aucResults$CRCvsCont_AUC[temp]=1 - aucResults$CRCvsCont_AUC[temp]
        }






#sorting results by the AA AUC
    if(sortBy=="AA"){
    
    aucResults=aucResults[order(aucResults$AAvsCont_AUC, decreasing=TRUE), ]
    }

    ##Sorting the results by the CRC AUC
    if(sortBy=="CRC"){
    
    aucResults=aucResults[order(aucResults$CRCvsCont_AUC, decreasing=TRUE), ]
    }


    return(aucResults)

}


getClassificationResults<-function(dataSet, 
                                   genesTested=genesTested_Omega_POC1,
                                   topGenes=c("HBA", "OLR1", "TGFBI", "CXCL8"),
                                   ControlGenes=c("Control", "HP"), 
                                   normalized=FALSE,
                                   numberFolds=5, 
                                   numberSingleGeneRepeats=1,
                                   numberCombinedRepeats=10,
                                   runSingleGenes=FALSE)
    {


#########extracting data and getting it in a form suitable for the tidymodels machine learning function calls
    if(normalized==FALSE)
    {
        rawCT=dataSet[, c(which(colnames(dataSet)=="Sample_Type" | colnames(dataSet)=="HB" | colnames(dataSet)=="TF"),
                         genesTested)]
        subsetData = rawCT %>% filter(rawCT[, 1] %in% c("CRC", "AA", ControlGenes))
    }

    if(normalized==TRUE){

    subsetData = dataSet %>% filter(dataSet[, 1] %in% c("CRC", "AA", ControlGenes))
    
    }
    

    subsetData = subsetData[, c(1, which(colnames(subsetData) %in% topGenes))]



#Initizliaing data structure that will hold the results
    CrossV_Results_combined=data.frame("Genes_Used"=c(colnames(subsetData)[2:ncol(subsetData)], "Combined_Genes"), 
                          "crcAUC"=rep(NA, ncol(subsetData)), 
                          "aaAUC"=rep(NA, ncol(subsetData)),
                          "CRCsensitivity"=rep(NA, ncol(subsetData)), 
                          "AAsensitivity"=rep(NA, ncol(subsetData)), 
                          "Specificity"=rep(NA, ncol(subsetData)))

##This will set the for loop that runs single genes classifiation performance 
##to never run i.e. only get classification performance for the combined genes
    if(runSingleGenes==FALSE){
        numGenesToRun=2
    }

    ##This will set the for loop that runs single genes classifiation performance to run for every single gene
    ##And to get the combined performance
    if(runSingleGenes==TRUE){
        numGenesToRun=ncol(subsetData)
    }

    ##Loop through all individual genes getting single gene classification performance
    for(i in 2:numGenesToRun){
        results=runCrossValidation(subsetData[, c(1, i)], controlGroup = ControlGenes,
                               numberFolds=numberFolds, repeats=numberSingleGeneRepeats)

    
 

        temp=collect_predictions(results, summarize = TRUE)
        temp$Label_Breakdown=subsetData$Sample_Type[temp$.row]
 
    

    #calculate sensitivity and specificity for all possible class probability cutoffs 
        results=calculateSensandSpecForSingleModel(temp, controlCategories=ControlGenes)


   
    
    ##Getting results using a 50%  disease class probability cutoff i.e. if disease class probability
    #is > 0.5 it is classified as disease if it is < 0.5 it is classified as control
        index=which(results$classProbCutoffs == 0.5)

   
    
   
        CrossV_Results_combined$CRCsensitivity[i-1]=results$CRC_Sens[index]
        CrossV_Results_combined$AAsensitivity[i-1]=results$AA_Sens[index]
        CrossV_Results_combined$Specificity[i-1]=results$Specificity[index]

    
    ##getting auc for CRC
        crcModelResults=temp[which(temp$Label_Breakdown %in% c("CRC", ControlGenes)), ]
        crcModelResults=crcModelResults[order(crcModelResults$.pred_Disease), ]
        crcModelResults=as.data.frame(crcModelResults)  
        aucSingleGene=auc(crcModelResults[, ".pred_Disease"], response=crcModelResults$Sample_labels, quiet=TRUE)


    
        CrossV_Results_combined$crcAUC[i-1]=aucSingleGene
    

    
    ##getting auc for AA
        aaModelResults=temp[which(temp$Label_Breakdown %in% c("AA", ControlGenes)), ]
        aaModelResults=aaModelResults[order(aaModelResults$.pred_Disease), ]
        aaModelResults=as.data.frame(aaModelResults)  
        aucSingleGene=auc(aaModelResults[, ".pred_Disease"], response=aaModelResults$Sample_labels, quiet=TRUE)

   
        CrossV_Results_combined$aaAUC[i-1]=aucSingleGene
    
}    


#getting classification results for all biomarkers
    results=runCrossValidation(subsetData, controlGroup = ControlGenes,
                               numberFolds=numberFolds, repeats=numberCombinedRepeats)
    temp=collect_metrics(results, summarize = TRUE)



    allGenesCombined=collect_predictions(results, summarize = TRUE)



    allGenesCombined$Label_Breakdown=subsetData$Sample_Type[allGenesCombined$.row]

#calculate sensitivity and specificity for all possible class probability cutoffs 
    results=calculateSensandSpecForSingleModel(allGenesCombined, controlCategories=ControlGenes)

##Getting AUC for combined genes
##getting auc for CRC
    crcModelResults=allGenesCombined[which(allGenesCombined$Label_Breakdown %in% c("CRC", ControlGenes)), ]
    crcModelResults=crcModelResults[order(crcModelResults$.pred_Disease), ]
    crcModelResults=as.data.frame(crcModelResults)  
    aucSingleGene=auc(crcModelResults[, ".pred_Disease"], response=crcModelResults$Sample_labels, quiet=TRUE)


    CrossV_Results_combined$crcAUC[nrow(CrossV_Results_combined)]=aucSingleGene


##getting auc for AA for combined genes
    aaModelResults=allGenesCombined[which(allGenesCombined$Label_Breakdown %in% c("AA", ControlGenes)), ]
    aaModelResults=aaModelResults[order(aaModelResults$.pred_Disease), ]
    aaModelResults=as.data.frame(aaModelResults)  
    aucSingleGene=auc(aaModelResults[, ".pred_Disease"], response=aaModelResults$Sample_labels, quiet=TRUE)

    CrossV_Results_combined$aaAUC[nrow(CrossV_Results_combined)]=aucSingleGene



###getting sensitivity and specificity for combined genes using 50% class probability cutoff
    index=which(results$classProbCutoffs==0.5)

    CrossV_Results_combined$CRCsensitivity[nrow(CrossV_Results_combined)]=results$CRC_Sens[index]
    CrossV_Results_combined$AAsensitivity[nrow(CrossV_Results_combined)]=results$AA_Sens[index]
    CrossV_Results_combined$Specificity[nrow(CrossV_Results_combined)]=results$Specificity[index]



    CrossV_Results_combined=CrossV_Results_combined

    
    return(CrossV_Results_combined)



}